In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import root_mean_squared_error,mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from time import time
import numpy as np

from utils.pipeline import create_pipeline
from utils.data_cleaning import load_and_clean

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = load_and_clean(verbose=True)

In [ ]:
X = df.drop('price', axis=1) #Features

y = df['price'] #Target

In [ ]:
model = KNeighborsRegressor() #Create linear regression model
pipeline = create_pipeline(df, model=model) #Create pipeline for linear regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Split data into training and testing sets

In [ ]:
# Fit the pipeline and predict
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("y shape:", y_test.shape)
print("y_pred shape:", y_pred.shape)

In [ ]:
#Evaluation
print(f"""
      Values of Target 'price':
      {'-' * 80}
      predicted y values:
      {y_pred.tolist()[:10]}

      actual y values:
      {y_test.tolist()[:10]}



      Model Performance Metrics:
      {'-' * 80}
      {'mean squared error:': <30} {mean_squared_error(y_test, y_pred):.5f}
      {'root mean squared error:': <30} {root_mean_squared_error(y_test, y_pred):.5f}
      {'mean absolute error:': <30} {mean_absolute_error(y_test, y_pred):.5f}
      {'r2 score:': <30} {r2_score(y_test, y_pred):.5f}
      """)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#calculate residuals
residuals = y_test - y_pred

#plot residuals
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals, alpha=0.5)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()


In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs Predicted Values")
plt.show()

In [ ]:
# Plot distribution of residuals
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.xlabel("Residuals")
plt.title("Distribution of Residuals")
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve
import numpy as np

In [ ]:
# Generate learning curve
train_sizes, train_scores, test_scores = learning_curve(
    pipeline, X_train, y_train, cv=5, scoring='r2', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
)

# Calculate mean and standard deviation
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label="Training Score", color="blue")
plt.plot(train_sizes, test_mean, label="Cross-Validation Score", color="orange")
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="blue", alpha=0.2)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="orange", alpha=0.2)
plt.xlabel("Training Set Size")
plt.ylabel("R² Score")
plt.title("Learning Curve")
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the hyperparameters to tune
param_grid = {
    'model_selection__n_neighbors': [3, 5, 7, 9, 11, 15],  # Number of neighbors
    'model_selection__weights': ['uniform', 'distance'],  # Weighting method
    'model_selection__p': [1, 2]  # Distance metric (Manhattan or Euclidean)
}

# Create a GridSearchCV object
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='r2',  # Metric to optimize
    n_jobs=-1  # Use all available cores
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best R² Score:", grid_search.best_score_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define the hyperparameters to tune
param_distributions = {
    'model_selection__n_neighbors': np.arange(1, 30),  # Randomly sample neighbors
    'model_selection__weights': ['uniform', 'distance'],  # Weighting method
    'model_selection__p': [1, 2]  # Distance metric
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=50,  # Number of random combinations to try
    cv=5,  # 5-fold cross-validation
    scoring='r2',  # Metric to optimize
    n_jobs=-1,
    random_state=42
)

# Perform the random search
random_search.fit(X, y)

# Print the best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best R² Score:", random_search.best_score_)

In [ ]:
# Get the best model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate performance
from sklearn.metrics import mean_squared_error, r2_score
print("Test R² Score:", r2_score(y_test, y_pred))
print("Test MSE:", mean_squared_error(y_test, y_pred))

In [ ]:
from joblib import dump
dump(best_model, 'knn.joblib')